In [1]:
import xarray as xr
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import geopandas as gpd

/home/mitch/mamba/envs/pygeo/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [2]:
import os
DATA_DIR = "/home/mitch/data/geog256"
os.listdir(DATA_DIR)

['dfmc_realigned.zarr',
 'era5_derived_hourly.zarr',
 'era5_daily.tar.gz',
 'era5_landsea_mask.grib',
 'era5_hourly.tar.gz',
 'dfmc.tar.gz',
 'era5_los_angeles.zarr',
 'era5_derived_daily.zarr',
 'dfmc_input.zarr',
 'perimeters',
 'dfmc.zarr',
 'era5_landsea_mask.grib.5b7b6.idx']

In [3]:
os.listdir(f"{DATA_DIR}/perimeters")

['Perimeters.shx',
 'WFIGS_Interagency_Perimeters_Current_-2078219868963267088.geojson',
 'Perimeters.shp',
 'Perimeters.cpg',
 'Perimeters.prj',
 'Perimeters.dbf']

In [4]:
import os
os.environ["PROJ_LIB"] = "/home/mitch/mamba/envs/pygeo/lib/python3.12/site-packages/pyproj/proj_dir/share/proj"


gdf = gpd.read_file(f"{DATA_DIR}/perimeters/WFIGS_Interagency_Perimeters_Current_-2078219868963267088.geojson")
print(gdf.head())
gdf.plot()

CRSError: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: no database context specified)

In [6]:

LA_BOUNDING_BOX = {
    "lat_min": 33.5,  # Southernmost point
    "lat_max": 34.8,  # Northernmost point
    "lon_min": -119.0, # Westernmost point
    "lon_max": -117.5  # Easternmost point
}

def filter_dataset_by_bbox(ds, lat_min, lat_max, lon_min, lon_max):
    return ds.sel(latitude=slice(lat_min, lat_max), longitude=slice(lon_min, lon_max))

def convert_longitude(ds):
    ds = ds.assign_coords(longitude=((ds["longitude"] + 180) % 360 - 180)) 
    ds = ds.sortby("longitude")  
    ds = ds.sortby("latitude")
    return ds

def apply_custom_mask(ds):
    exclude_coords = [
        (33.78, -118.9),
        (33.53, -118.9),
        (33.78, -118.65),
        (33.53, -118.65),
        (33.53, -118.4),
        (33.53, -118.15),
        (33.53, -117.9),
    ]
    for lat, lon in exclude_coords:
        ds = ds.where(~((ds.latitude == lat) & (ds.longitude == lon)), drop=True)
    return ds




In [13]:
era5_ds = xr.open_dataset(f"{DATA_DIR}/era5_derived_hourly.zarr", chunks="auto").sortby("latitude")
era5_ds = filter_dataset_by_bbox(
    era5_ds,
    **LA_BOUNDING_BOX,
)
era5_ds = apply_custom_mask(era5_ds)

wind_mean = era5_ds["wind"].resample(valid_time="1D").mean()
avg_u10 = era5_ds["u10"].resample(valid_time="1D").mean()
avg_v10 = era5_ds["v10"].resample(valid_time="1D").mean()
wind = era5_ds["wind"].resample(valid_time="1D").mean()
ffwi = era5_ds["fosberg"].resample(valid_time="1D").mean()

e5_daily_ds = xr.Dataset({
    "wind_mean": wind,
    "u10": avg_u10,
    "v10": avg_v10,
    "wind": wind,
    "ffwi": ffwi,
})
e5_daily_ds

<xarray.Dataset> Size: 20MB
Dimensions:     (latitude: 6, longitude: 6, valid_time: 27042)
Coordinates:
  * latitude    (latitude) float64 48B 33.53 33.78 34.03 34.28 34.53 34.78
  * longitude   (longitude) float64 48B -118.9 -118.7 -118.4 ... -117.9 -117.7
  * valid_time  (valid_time) datetime64[ns] 216kB 1951-01-01 ... 2025-01-13
Data variables:
    wind_mean   (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1792, 6, 6), meta=np.ndarray>
    u10         (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1792, 6, 6), meta=np.ndarray>
    v10         (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1792, 6, 6), meta=np.ndarray>
    wind        (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1792, 6, 6), meta=np.ndarray>
    ffwi        (valid_time, latitude, longitude) float32 4MB dask.array<chunksize=(1792, 6, 6), meta=np.ndarray>